In [1]:
import cv2
import face_recognition as fr
import numpy as np
import os

# Load the pre-trained face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

faces_path = "C:\\Users\\LAPTOP\\Pictures\\Camera Roll\\known"

def get_face_encodings():
    face_names = os.listdir(faces_path)
    face_encodings = []

    # For loop to retrieve all face encodings and store them in a list.
    # Below loop also gets the names of people and removes ".jpg", and stores
    # the names in a list
    for i, name in enumerate(face_names):
        face = fr.load_image_file(f"{faces_path}\\{name}")
        face_encodings.append(fr.face_encodings(face)[0])

        face_names[i] = name.split(".")[0] # To remove ".jpg" or any other image extension

    return face_encodings, face_names

face_encodings, face_names = get_face_encodings()

# Initialize the webcam
video_capture = cv2.VideoCapture(0)
scl = 2
name = ""
name2=""
while True:
    # Capture frame-by-frame from the webcam
    ret, frame = video_capture.read()

    # Making current frame smaller so the program runs faster
    resized_frame = cv2.resize(frame, (int(frame.shape[1]/scl), int(frame.shape[0]/scl)))

    # Convert the frame to RGB for face recognition
    rgb_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)

    # Detect faces in the grayscale frame
    faces = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Retrieve face location coordinates and unknown encodings
    face_locations = fr.face_locations(rgb_frame)
    unknown_encodings = fr.face_encodings(rgb_frame, face_locations)

    # Get the number of detected faces
    num_faces = len(face_locations)

    # Iterating through each encoding, as well as the face's location
    for face_encoding, face_location in zip(unknown_encodings, face_locations):
        # Comparing known faces with unknown faces
        result = fr.compare_faces(face_encodings, face_encoding, 0.4)

        # Getting the correct name if a match was found
        if True in result:
            name = face_names[result.index(True)]
            name2 = face_names[result.index(True)]
        else:
            # If no match is found, print "unknown"
            name = "unknown"

        # Setting coordinates for face location
        top, right, bottom, left = face_location

        # Drawing a rectangle around the face
        cv2.rectangle(frame, (left*scl, top*scl), (right*scl, bottom*scl), (0, 0, 255), 2)

        # Setting font, as well as displaying the text of the name and number of people
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, f"Number of people: {num_faces}", (10, 30), font, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, name, (left*scl, bottom*scl + 20), font, 0.8, (255, 255, 255), 1)

    # Displaying the resulting frame with face detections
    cv2.imshow('Webcam', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all OpenCV windows
video_capture.release()
cv2.destroyAllWindows()
